# Step-by-step implementation
The following are the steps to implement the decomposition technique:
1. Import necessary libraries
2. Set up the LangSmith and OpenAI API keys
3. Load document
4. Split text
5. Index documents
6. Create sub-questions: Decompose the main question
7. Generate answers for sub-questions
8. Merge sub-questions and answers
9. Generate the final answer

## 1. Import necessary libraries

In [0]:
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

## 2. Set up the LangSmith and OpenAI API keys

In [0]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] = ""  # Add your OpenAI API key
if OPENAI_API_KEY == "":
    raise ValueError("Please set the OPENAI_API_KEY environment variable")

In [0]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '' # Add your LangSmith LangChain API key
os.environ['LANGCHAIN_PROJECT']='Decomposition'

## 3. Load document

In [0]:
# provide you document
loader = WebBaseLoader(
    web_paths=("https://blog.langchain.dev/announcing-langsmith",),
)

blog_docs = loader.load()

## 4. Split text

In [0]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100,
    chunk_overlap=20)

splits = text_splitter.split_documents(blog_docs)

## 5. Index documents

In [0]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

## 6. Create sub-questions: Decompose the main question

In [0]:
template = """You are an AI language model assistant that generates multiple sub-questions related to an input question. \n
Your task is to break down the input into three sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Original question: {question}"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(temperature=0)

generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

question = "What is LangSmith, and why do we need it?"

generate_queries_decomposition.invoke({"question":question})

## 7. Generate answers for sub-questions

In [0]:
prompt_rag = hub.pull("rlm/rag-prompt")

sub_questions = generate_queries_decomposition.invoke({"question":question})
rag_results = []
for sub_question in sub_questions:
  retrieved_docs = retriever.invoke(sub_question)
  answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs,
                                                                "question": sub_question})
  rag_results.append(answer)

## 8. Merge sub-questions and answers

In [0]:
def format(questions, answers):
    """Format Q and A"""

    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format(sub_questions, rag_results)

## 9. Generate the final answer

In [0]:
template = """Here is a set of Q and A:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})